In [1]:
#Импортируем библиотеки
import numpy as np
import pandas as pd
import copy
import re
import openpyxl

In [2]:
df = pd.read_csv("C:/Users/NightMare/Desktop/YandexHack/DATA_Pretrained/2020_new_обработанный_2.csv", encoding='Windows-1251')

In [3]:
df.head(15)

,Направление расходов,Наименование вида расходов,Вид объекта,Дата проведения,Конечный остаток
0,"Калинина,261",ТПЭС,Здание,31.12.2020,32790.96
1,"Седова, 23",ЭЭ,Здание,31.12.2020,2274.13
2,"Светланская,111",ЭЭ,Здание,31.12.2020,238482.63
3,"Находка, Находкинский пр-т, 13",ЭЭ,Здание,31.12.2020,614813.95
4,"Фонтанная,19",ЭЭ,Здание,31.12.2020,22413.58
5,"Семеновская,29а",ЭЭ,Здание,31.12.2020,4397.30
6,"Калинина,261",ЭЭ,Здание,31.12.2020,307392.53
7,"Мордовцева,6",ЭЭ,Здание,31.12.2020,157352.26
8,"Светланская,69",ЭЭ,Здание,31.12.2020,367137.39
9,"Светланская,71",ЭЭ,Здание,31.12.2020,1781068.95


In [4]:
def remove_whitespace(column):
    return column.str.replace(' ', '')
def remove_suffix(column):
    return column.str.rstrip(',-')

df['Направление расходов'] = remove_whitespace(df['Направление расходов'])
df['Направление расходов'] = remove_suffix(df['Направление расходов'])

In [5]:
def compare_and_assign(df, column_name, replacements):
    for index, value in df[column_name].items():
        for replacement in replacements:
            target_string = replacement['target']
            new_value = replacement['new_value']
            # Удаляем пробелы с обеих сторон строки перед проверкой
            if value.rstrip().endswith(target_string):
                df.at[index, "Вид объекта"] = new_value
                # Удаляем target строку из строки с которой сравнивали
                df.at[index, column_name] = value.replace(target_string, '').strip()
    return df[column_name]  # Возвращаем измененный столбец

# Заменяем значения, оканчивающиеся на "(Буфет)"
replacements = [
    {'target': ',кв.30', 'new_value': ''}
]
df['Направление расходов'] = compare_and_assign(df, 'Направление расходов', replacements)



def compare_and_assign_prefix(df, column_name, replacements):
    for index, value in df[column_name].items():
        for replacement in replacements:
            target_string = replacement['target']
            new_value = replacement['new_value']
            # Удаляем пробелы с обеих сторон строки перед проверкой
            if value.lstrip().startswith(target_string):
                df.at[index, "Вид объекта"] = new_value
                # Удаляем target строку из строки с которой сравнивали
                df.at[index, column_name] = value.replace(target_string, '', 1).strip()
    return df[column_name]  # Возвращаем измененный столбец

replacements = [
    {'target': 'РКЦВилючинск,', 'new_value': ''},
    {'target': 'Благовещенск,', 'new_value': ''},
    {'target': 'Южно-Сахалинск,', 'new_value': ''},
    {'target': 'РКЦКомсомольск-на-Амуре', 'new_value': ''},
    {'target': 'ул,', 'new_value': ''},
    {'target': 'мкр.', 'new_value': ''},
    {'target': 'пр.', 'new_value': ''},
    {'target': 'РКЦ', 'new_value': ''},
    {'target': 'п.Депутатский,мкр.', 'new_value': ''},
    {'target': 'Находка,', 'new_value': ''},
    {'target': 'пер.', 'new_value': ''},
    {'target': 'Ленск', 'new_value': ''},
    {'target': 'Покровск', 'new_value': ''},
    {'target': 'Нерюнгри', 'new_value': ''},
    {'target': 'Южная', 'new_value': ''},
    {'target': '50летСоветскойАрмии', 'new_value': ''},
    {'target': 'Билибино', 'new_value': ''},
    {'target': 'Комсомольск-на-Амуре', 'new_value': ''},
    {'target': ',', 'new_value': ''},
    {'target': ' Профсоюзная,д.,12', 'new_value': ''},
    {'target': 'Зея,пер.2', 'new_value': ''},
    {'target': 'кв.29', 'new_value': ''},
    {'target': 'КарлаМаркса,,,д.16', 'new_value': ''},
    {'target': 'Горького7', 'new_value': ''},
    {'target': 'Биробиджан,', 'new_value': ''}   
]
df['Направление расходов'] = compare_and_assign_prefix(df, 'Направление расходов', replacements)


def drop_rows_with_empty_values(df):
    # Удаляем строки, содержащие хотя бы одно из значений: 0, NaN, пустую строку или пробелы
    df = df.dropna().replace({'': None, ' ': None}).dropna()
    return df

# Удаляем строки с пустыми значениями
df= drop_rows_with_empty_values(df)



def add_comma_before_d(column):
    # Применяем функцию к каждой ячейке столбца
    column_with_comma = column.apply(lambda x: x[:x.find('д.')] + ',' + x[x.find('д.'):] if 'д.' in x else x)
    
    return column_with_comma

df['Направление расходов'] = add_comma_before_d(df['Направление расходов'])


# Выведите уникальные значения
for value in df['Направление расходов'].unique():
    print(value)

Калинина,261
Седова,23
Светланская,111
Фонтанная,19
Семеновская,29а
Мордовцева,6
Светланская,69
Светланская,71
Светланская,73
Светланская,73А
Океанскийпр-т,34
Пушкинская,8Б
Некрасова,102
Пушкина,59
Ленина,64
Пушкинская,8б
пр-т50летОктября,89
Дежнева,7
Колхозная32Б
п.Эгвекинот
Горького,7
Дежнева7
пгт.Эгвекинот
Чапаева,5А
БогданаХмельницкого52/2
Профсоюзная,,д.12
Ленина,29
Зеяпер.Жуковский,15
Профсоюзная,12
Ленина29
Зея,пер.Жуковский,15
БогданаХмельницкого,52/2
Пушкина,4
Пушкина,6
К.Маркса,36-20,кв.29
К.Маркса36-20,кв.29
Поротова,14
Ленинская,33
Партизанская,11
Ленинская,22
Ленинская22
Советская,26
Воронежская,154
М-Амурского,42
М-Амурского,21
Калинина,95
Советская,27
КарлаМаркса,,д.16
Кирова,17
Ленина,70
Ленина,22
Верхнепортовая,44
Муравьева-Амурского,21
Муравьева-Амурского,42


In [6]:
df.head()

,Направление расходов,Наименование вида расходов,Вид объекта,Дата проведения,Конечный остаток
0,"Калинина,261",ТПЭС,Здание,31.12.2020,32790.96
1,"Седова,23",ЭЭ,Здание,31.12.2020,2274.13
2,"Светланская,111",ЭЭ,Здание,31.12.2020,238482.63
4,"Фонтанная,19",ЭЭ,Здание,31.12.2020,22413.58
5,"Семеновская,29а",ЭЭ,Здание,31.12.2020,4397.30


In [7]:
def split_date_column(df, date_column_name):
    df[date_column_name] = pd.to_datetime(df[date_column_name], format='%d.%m.%Y')
    df['Год проведения'] = df[date_column_name].dt.year
    df['Месяц проведения'] = df[date_column_name].dt.month
    df.drop(columns=[date_column_name], inplace=True)
    return df

df = split_date_column(df, 'Дата проведения')

df.head(15)

,Направление расходов,Наименование вида расходов,Вид объекта,Конечный остаток,Год проведения,Месяц проведения
0,"Калинина,261",ТПЭС,Здание,32790.96,2020,12
1,"Седова,23",ЭЭ,Здание,2274.13,2020,12
2,"Светланская,111",ЭЭ,Здание,238482.63,2020,12
4,"Фонтанная,19",ЭЭ,Здание,22413.58,2020,12
5,"Семеновская,29а",ЭЭ,Здание,4397.30,2020,12
6,"Калинина,261",ЭЭ,Здание,307392.53,2020,12
7,"Мордовцева,6",ЭЭ,Здание,157352.26,2020,12
8,"Светланская,69",ЭЭ,Здание,367137.39,2020,12
9,"Светланская,71",ЭЭ,Здание,1781068.95,2020,12
10,"Светланская,71",ЭЭ,пристройка,530562.51,2020,12


In [8]:
# Переносим Конечный остаток вправо.
new_order = [col for col in df.columns if col != 'Конечный остаток'] + ['Конечный остаток']
df = df[new_order]

In [9]:
# Группировка по другим столбцам и суммирование 'конечный остаток'
df = df.groupby(['Направление расходов', 'Наименование вида расходов', 'Год проведения', 'Месяц проведения'])['Конечный остаток'].sum().reset_index()

In [10]:
df.head()

,Направление расходов,Наименование вида расходов,Год проведения,Месяц проведения,Конечный остаток
0,"БогданаХмельницкого,52/2",ВО,2020,12,39.04
1,"БогданаХмельницкого,52/2",ХВС,2020,12,41.66
2,БогданаХмельницкого52/2,ТЭ,2020,12,3421.83
3,БогданаХмельницкого52/2,ЭЭ,2020,12,1178654.71
4,"Верхнепортовая,44",ВО,2020,12,8413.00


In [11]:
len(df)

155

In [12]:
# Сохранение файла в формате CSV с указанием кодировки UTF-8
df.to_csv('C:/Users/NightMare/Desktop/YandexHack/DATA_well_done/2020_new_welldone.csv', index=False, encoding='Windows-1251')

# Сохранение в Excel
df.to_excel('C:/Users/NightMare/Desktop/YandexHack/DATA_well_done/2020_new_welldone.xlsx', index=False)